# Credit Scoring con Deep Learning
## Red Neuronal para Predicción de Incumplimiento de Tarjetas de Crédito

Este notebook implementa un modelo de Deep Learning para calcular el score crediticio utilizando una red neuronal artificial (ANN).

## 1. Importar Librerías

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.metrics import confusion_matrix, classification_report, roc_curve

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam

import warnings
warnings.filterwarnings('ignore')

print(f"TensorFlow version: {tf.__version__}")
print(f"GPU disponible: {tf.config.list_physical_devices('GPU')}")

## 2. Cargar y Explorar los Datos

In [ ]:
# Cargar el dataset
# NOTA: Ajusta la ruta según la ubicación de tu archivo
df = pd.read_csv('/content/drive/MyDrive/Credit Model/Credit_Card_Dataset.csv')

print("Dimensiones del dataset:", df.shape)
print("\nPrimeras filas:")
df.head()

In [ ]:
# Información general del dataset
print("\nInformación del dataset:")
df.info()

print("\nEstadísticas descriptivas:")
df.describe()

In [ ]:
# Verificar valores nulos
print("\nValores nulos por columna:")
print(df.isnull().sum())

# Distribución de la variable objetivo
print("\nDistribución de la variable 'Defaulted':")
print(df['Defaulted'].value_counts())
print(f"\nPorcentaje de incumplimiento: {df['Defaulted'].mean()*100:.2f}%")

## 3. Preprocesamiento de Datos

In [ ]:
# Crear una copia del dataframe para preprocesamiento
df_processed = df.copy()

# Eliminar Customer_ID ya que es un identificador único
df_processed = df_processed.drop('Customer_ID', axis=1)

print("Columnas del dataset:", df_processed.columns.tolist())
print(f"\nNúmero de características: {df_processed.shape[1]-1}")

In [ ]:
# Identificar columnas categóricas y numéricas
categorical_cols = df_processed.select_dtypes(include=['object']).columns.tolist()
numerical_cols = df_processed.select_dtypes(include=['int64', 'float64']).columns.tolist()

# Remover 'Defaulted' de las columnas numéricas (es la variable objetivo)
if 'Defaulted' in numerical_cols:
    numerical_cols.remove('Defaulted')

print(f"Columnas categóricas ({len(categorical_cols)}): {categorical_cols}")
print(f"\nColumnas numéricas ({len(numerical_cols)}): {numerical_cols}")

In [ ]:
# Codificar variables categóricas usando Label Encoding
label_encoders = {}

for col in categorical_cols:
    le = LabelEncoder()
    df_processed[col] = le.fit_transform(df_processed[col])
    label_encoders[col] = le
    print(f"{col}: {len(le.classes_)} categorías")

print("\nCodificación completada.")

In [ ]:
# Separar características (X) y variable objetivo (y)
X = df_processed.drop('Defaulted', axis=1)
y = df_processed['Defaulted']

print(f"Forma de X: {X.shape}")
print(f"Forma de y: {y.shape}")
print(f"\nNúmero total de características: {X.shape[1]}")

In [ ]:
# Dividir en conjuntos de entrenamiento y prueba (80-20)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print(f"Tamaño del conjunto de entrenamiento: {X_train.shape[0]} ({X_train.shape[0]/len(X)*100:.1f}%)")
print(f"Tamaño del conjunto de prueba: {X_test.shape[0]} ({X_test.shape[0]/len(X)*100:.1f}%)")
print(f"\nDistribución en entrenamiento - Incumplimiento: {y_train.mean()*100:.2f}%")
print(f"Distribución en prueba - Incumplimiento: {y_test.mean()*100:.2f}%")

In [ ]:
# Configurar el scaler según hiperparámetrosfrom sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler, MaxAbsScalerif hiperparametros['scaler_type'] == 'standard':    scaler = StandardScaler()elif hiperparametros['scaler_type'] == 'minmax':    scaler = MinMaxScaler(feature_range=hiperparametros['scaler_feature_range'])elif hiperparametros['scaler_type'] == 'robust':    scaler = RobustScaler()elif hiperparametros['scaler_type'] == 'maxabs':    scaler = MaxAbsScaler()else:    scaler = StandardScaler()X_train_scaled = scaler.fit_transform(X_train)X_test_scaled = scaler.transform(X_test)print(f"Escalado completado usando: {hiperparametros['scaler_type']}")print(f"\nMedia de características (train): {X_train_scaled.mean():.6f}")print(f"Desviación estándar (train): {X_train_scaled.std():.6f}")

## 4. Construcción del Modelo de Deep Learning

In [ ]:
# Definir los hiperparámetroshiperparametros = {    'num_layers': 6,  # Número total de capas ocultas    'neurons': [500, 200, 100, 16],  # Neuronas por capa (se repite el último si faltan)    'activation_hidden': 'relu',      'activation_output': 'sigmoid',      'kernel_initializer': 'he_uniform',          'dropout_rates': [0.3, 0.3, 0.2, 0.2, 0.2, 0.2],  # Dropout por capa    'use_batch_norm': True,      'batch_norm_momentum': 0.99,      'batch_norm_epsilon': 0.001,      'l1_regularization': 0.0,  # L1 regularization (0.0 = desactivado)    'l2_regularization': 0.01,  # L2 regularization (0.0 = desactivado)    'kernel_constraint': None,  # None o valor para max_norm        'optimizer_type': 'adam',      'learning_rate': 0.001,      'beta_1': 0.9,  # Para Adam/Nadam    'beta_2': 0.999,  # Para Adam/Nadam    'epsilon': 1e-07,  # Para Adam    'amsgrad': False,  # Usar AMSGrad variant    'momentum': 0.0,  # Para SGD (si se usa)    'nesterov': False,  # Usar Nesterov momentum (para SGD)    'clipnorm': None,  # Gradient clipping por norma (None = desactivado)    'clipvalue': None,  # Gradient clipping por valor (None = desactivado)        'loss': 'binary_crossentropy',      'focal_loss_gamma': 2.0,      'focal_loss_alpha': 0.25,         'metrics': ['accuracy', 'auc', 'precision', 'recall'],         'epochs': 300,      'batch_size': 50,     'shuffle': True,  # Mezclar datos en cada época        'use_early_stopping': True,    'early_stopping_monitor': 'val_loss',  # 'val_loss', 'val_accuracy', 'val_auc'    'early_stopping_patience': 15,    'early_stopping_min_delta': 0.0001,    'early_stopping_mode': 'auto',      'restore_best_weights': True,        'use_reduce_lr': True,    'reduce_lr_monitor': 'val_loss',    'reduce_lr_factor': 0.5,    'reduce_lr_patience': 5,    'reduce_lr_min_lr': 0.00001,    'reduce_lr_min_delta': 0.0001,    'scaler_type': 'standard',  # 'standard', 'minmax', 'robust', 'maxabs'    'scaler_feature_range': (0, 1),  # Para MinMaxScaler        'verbose': 1,      'prediction_threshold': 0.5, }# Definir la arquitectura de la red neuronal con hiperparámetros configurablesdef create_deep_learning_model(input_dim, hp):    """    Crea un modelo de red neuronal profunda para clasificación binaria.        Args:        input_dim: Dimensión de entrada (número de características)        hp: Diccionario con hiperparámetros        Returns:        Modelo compilado de Keras    """    from tensorflow.keras import regularizers    from tensorflow.keras.constraints import MaxNorm        model = Sequential()        # Extender la lista de neuronas si es necesario    neurons_list = hp['neurons'].copy()    if len(neurons_list) < hp['num_layers']:        last_neurons = neurons_list[-1]        neurons_list.extend([last_neurons] * (hp['num_layers'] - len(neurons_list)))        # Extender la lista de dropout si es necesario    dropout_list = hp['dropout_rates'].copy()    if len(dropout_list) < hp['num_layers']:        dropout_list.extend([dropout_list[-1]] * (hp['num_layers'] - len(dropout_list)))        # Configurar regularización    kernel_reg = None    if hp['l1_regularization'] > 0 and hp['l2_regularization'] > 0:        kernel_reg = regularizers.l1_l2(l1=hp['l1_regularization'], l2=hp['l2_regularization'])    elif hp['l1_regularization'] > 0:        kernel_reg = regularizers.l1(hp['l1_regularization'])    elif hp['l2_regularization'] > 0:        kernel_reg = regularizers.l2(hp['l2_regularization'])        # Configurar constraint    kernel_const = MaxNorm(hp['kernel_constraint']) if hp['kernel_constraint'] else None        # Crear capas ocultas    for i in range(hp['num_layers']):        if i == 0:            # Primera capa (entrada)            model.add(Dense(                neurons_list[i],                activation=hp['activation_hidden'],                input_dim=input_dim,                kernel_initializer=hp['kernel_initializer'],                kernel_regularizer=kernel_reg,                kernel_constraint=kernel_const,                name=f'layer_{i}'            ))        else:            # Capas ocultas            model.add(Dense(                neurons_list[i],                activation=hp['activation_hidden'],                kernel_initializer=hp['kernel_initializer'],                kernel_regularizer=kernel_reg,                kernel_constraint=kernel_const,                name=f'layer_{i}'            ))                # Batch Normalization        if hp['use_batch_norm']:            model.add(BatchNormalization(                momentum=hp['batch_norm_momentum'],                epsilon=hp['batch_norm_epsilon']            ))                # Dropout        if dropout_list[i] > 0:            model.add(Dropout(dropout_list[i]))        # Capa de salida    model.add(Dense(        1,        activation=hp['activation_output'],        name='output_layer'    ))        # Configurar optimizador    if hp['optimizer_type'].lower() == 'adam':        optimizer = tf.keras.optimizers.Adam(            learning_rate=hp['learning_rate'],            beta_1=hp['beta_1'],            beta_2=hp['beta_2'],            epsilon=hp['epsilon'],            amsgrad=hp['amsgrad'],            clipnorm=hp['clipnorm'],            clipvalue=hp['clipvalue']        )    elif hp['optimizer_type'].lower() == 'sgd':        optimizer = tf.keras.optimizers.SGD(            learning_rate=hp['learning_rate'],            momentum=hp['momentum'],            nesterov=hp['nesterov'],            clipnorm=hp['clipnorm'],            clipvalue=hp['clipvalue']        )    elif hp['optimizer_type'].lower() == 'nadam':        optimizer = tf.keras.optimizers.Nadam(            learning_rate=hp['learning_rate'],            beta_1=hp['beta_1'],            beta_2=hp['beta_2'],            epsilon=hp['epsilon'],            clipnorm=hp['clipnorm'],            clipvalue=hp['clipvalue']        )    elif hp['optimizer_type'].lower() == 'rmsprop':        optimizer = tf.keras.optimizers.RMSprop(            learning_rate=hp['learning_rate'],            momentum=hp['momentum'],            epsilon=hp['epsilon'],            clipnorm=hp['clipnorm'],            clipvalue=hp['clipvalue']        )    else:        optimizer = Adam(learning_rate=hp['learning_rate'])        # Configurar métricas    metrics_list = []    for metric in hp['metrics']:        if metric == 'accuracy':            metrics_list.append('accuracy')        elif metric == 'auc':            metrics_list.append(tf.keras.metrics.AUC(name='auc'))        elif metric == 'precision':            metrics_list.append(tf.keras.metrics.Precision(name='precision'))        elif metric == 'recall':            metrics_list.append(tf.keras.metrics.Recall(name='recall'))        # Compilar el modelo    model.compile(        optimizer=optimizer,        loss=hp['loss'],        metrics=metrics_list    )        return model# Crear el modelo con los hiperparámetrosinput_dimension = X_train_scaled.shape[1]model = create_deep_learning_model(input_dimension, hiperparametros)# Mostrar la arquitectura del modeloprint("Arquitectura del Modelo de Deep Learning:\n")model.summary()

## 5. Entrenamiento del Modelo

In [ ]:
# Definir callbacks basados en hiperparámetroscallbacks = []if hiperparametros['use_early_stopping']:    early_stopping = EarlyStopping(        monitor=hiperparametros['early_stopping_monitor'],        patience=hiperparametros['early_stopping_patience'],        min_delta=hiperparametros['early_stopping_min_delta'],        mode=hiperparametros['early_stopping_mode'],        restore_best_weights=hiperparametros['restore_best_weights'],        verbose=hiperparametros['verbose']    )    callbacks.append(early_stopping)if hiperparametros['use_reduce_lr']:    reduce_lr = ReduceLROnPlateau(        monitor=hiperparametros['reduce_lr_monitor'],        factor=hiperparametros['reduce_lr_factor'],        patience=hiperparametros['reduce_lr_patience'],        min_lr=hiperparametros['reduce_lr_min_lr'],        min_delta=hiperparametros['reduce_lr_min_delta'],        verbose=hiperparametros['verbose']    )    callbacks.append(reduce_lr)print("Callbacks configurados:")if hiperparametros['use_early_stopping']:    print(f"- Early Stopping: detiene si no hay mejora en {hiperparametros['early_stopping_patience']} épocas")if hiperparametros['use_reduce_lr']:    print(f"- Reduce LR: reduce learning rate si no hay mejora en {hiperparametros['reduce_lr_patience']} épocas")

In [ ]:
# Entrenar el modelo con hiperparámetros configurablesprint("\nIniciando entrenamiento...\n")history = model.fit(    X_train_scaled, y_train,    validation_split=0.2,    epochs=hiperparametros['epochs'],    batch_size=hiperparametros['batch_size'],    shuffle=hiperparametros['shuffle'],    callbacks=callbacks,    verbose=hiperparametros['verbose'])print("\n¡Entrenamiento completado!")

## 6. Visualización del Entrenamiento

In [ ]:
# Visualizar las curvas de aprendizaje
def plot_training_history(history):
    fig, axes = plt.subplots(2, 2, figsize=(15, 10))
    
    # Loss
    axes[0, 0].plot(history.history['loss'], label='Training Loss', linewidth=2)
    axes[0, 0].plot(history.history['val_loss'], label='Validation Loss', linewidth=2)
    axes[0, 0].set_title('Pérdida durante el Entrenamiento', fontsize=14, fontweight='bold')
    axes[0, 0].set_xlabel('Época')
    axes[0, 0].set_ylabel('Loss')
    axes[0, 0].legend()
    axes[0, 0].grid(True, alpha=0.3)
    
    # Accuracy
    axes[0, 1].plot(history.history['accuracy'], label='Training Accuracy', linewidth=2)
    axes[0, 1].plot(history.history['val_accuracy'], label='Validation Accuracy', linewidth=2)
    axes[0, 1].set_title('Exactitud durante el Entrenamiento', fontsize=14, fontweight='bold')
    axes[0, 1].set_xlabel('Época')
    axes[0, 1].set_ylabel('Accuracy')
    axes[0, 1].legend()
    axes[0, 1].grid(True, alpha=0.3)
    
    # AUC
    axes[1, 0].plot(history.history['auc'], label='Training AUC', linewidth=2)
    axes[1, 0].plot(history.history['val_auc'], label='Validation AUC', linewidth=2)
    axes[1, 0].set_title('AUC durante el Entrenamiento', fontsize=14, fontweight='bold')
    axes[1, 0].set_xlabel('Época')
    axes[1, 0].set_ylabel('AUC')
    axes[1, 0].legend()
    axes[1, 0].grid(True, alpha=0.3)
    
    # Precision y Recall
    axes[1, 1].plot(history.history['precision'], label='Training Precision', linewidth=2)
    axes[1, 1].plot(history.history['val_precision'], label='Validation Precision', linewidth=2)
    axes[1, 1].plot(history.history['recall'], label='Training Recall', linewidth=2, linestyle='--')
    axes[1, 1].plot(history.history['val_recall'], label='Validation Recall', linewidth=2, linestyle='--')
    axes[1, 1].set_title('Precision y Recall durante el Entrenamiento', fontsize=14, fontweight='bold')
    axes[1, 1].set_xlabel('Época')
    axes[1, 1].set_ylabel('Métrica')
    axes[1, 1].legend()
    axes[1, 1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()

plot_training_history(history)

## 7. Evaluación del Modelo

In [ ]:
# Realizar predicciones con umbral configurabley_pred_proba = model.predict(X_test_scaled)y_pred = (y_pred_proba > hiperparametros['prediction_threshold']).astype(int).flatten()print("Predicciones realizadas en el conjunto de prueba.")print(f"Umbral de predicción: {hiperparametros['prediction_threshold']}")print(f"Forma de y_pred_proba: {y_pred_proba.shape}")print(f"Forma de y_pred: {y_pred.shape}")

In [ ]:
# Calcular métricas de evaluación
accuracy_dl = accuracy_score(y_test, y_pred)
precision_dl = precision_score(y_test, y_pred)
recall_dl = recall_score(y_test, y_pred)
f1_dl = f1_score(y_test, y_pred)
roc_auc_dl = roc_auc_score(y_test, y_pred_proba)

print("="*60)
print("MÉTRICAS DE RENDIMIENTO - MODELO DE DEEP LEARNING")
print("="*60)
print(f"Accuracy:  {accuracy_dl:.4f} ({accuracy_dl*100:.2f}%)")
print(f"Precision: {precision_dl:.4f}")
print(f"Recall:    {recall_dl:.4f}")
print(f"F1-Score:  {f1_dl:.4f}")
print(f"AUC-ROC:   {roc_auc_dl:.4f}")
print("="*60)

In [ ]:
# Reporte de clasificación detallado
print("\nREPORTE DE CLASIFICACIÓN DETALLADO:\n")
print(classification_report(y_test, y_pred, target_names=['No Incumplimiento', 'Incumplimiento']))

In [ ]:
# Matriz de confusión
cm = confusion_matrix(y_test, y_pred)

plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=['No Incumplimiento', 'Incumplimiento'],
            yticklabels=['No Incumplimiento', 'Incumplimiento'],
            cbar_kws={'label': 'Conteo'})
plt.title('Matriz de Confusión - Modelo Deep Learning', fontsize=14, fontweight='bold')
plt.ylabel('Valor Real')
plt.xlabel('Valor Predicho')
plt.tight_layout()
plt.show()

print("\nInterpretación de la Matriz de Confusión:")
print(f"Verdaderos Negativos (TN): {cm[0,0]} - Correctamente clasificados como NO incumplimiento")
print(f"Falsos Positivos (FP): {cm[0,1]} - Incorrectamente clasificados como incumplimiento")
print(f"Falsos Negativos (FN): {cm[1,0]} - Incorrectamente clasificados como NO incumplimiento")
print(f"Verdaderos Positivos (TP): {cm[1,1]} - Correctamente clasificados como incumplimiento")

In [ ]:
# Curva ROC
fpr, tpr, thresholds = roc_curve(y_test, y_pred_proba)

plt.figure(figsize=(10, 6))
plt.plot(fpr, tpr, color='darkorange', lw=2, 
         label=f'Curva ROC (AUC = {roc_auc_dl:.4f})')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--', label='Línea Base (AUC = 0.5)')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('Tasa de Falsos Positivos (FPR)', fontsize=12)
plt.ylabel('Tasa de Verdaderos Positivos (TPR)', fontsize=12)
plt.title('Curva ROC - Modelo Deep Learning', fontsize=14, fontweight='bold')
plt.legend(loc='lower right', fontsize=11)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## 8. Comparación de Modelos

In [ ]:
# Crear un DataFrame comparativo con los modelos anteriores
# NOTA: Estos valores son del modelo original de Random Forest
# Ajusta estos valores si tienes resultados diferentes

comparison_data = {
    'Modelo': ['Regresión Logística', 'Random Forest', 'Deep Learning'],
    'Accuracy': [0.6300, 0.6420, accuracy_dl],  # Valores aproximados del modelo original
    'Precision': [0.4500, 0.4637, precision_dl],
    'Recall': [0.3200, 0.3382, recall_dl],
    'F1-Score': [0.3750, 0.3912, f1_dl],
    'AUC-ROC': [0.6200, 0.6377, roc_auc_dl]
}

comparison_df = pd.DataFrame(comparison_data)
print("\nCOMPARACIÓN DE MODELOS:\n")
print(comparison_df.to_string(index=False))

# Identificar el mejor modelo por métrica
print("\n" + "="*60)
print("MEJOR MODELO POR MÉTRICA:")
print("="*60)
for metric in ['Accuracy', 'Precision', 'Recall', 'F1-Score', 'AUC-ROC']:
    best_idx = comparison_df[metric].idxmax()
    best_model = comparison_df.loc[best_idx, 'Modelo']
    best_value = comparison_df.loc[best_idx, metric]
    print(f"{metric:12} : {best_model:20} ({best_value:.4f})")

In [ ]:
# Visualización comparativa
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Gráfico de barras agrupadas
metrics = ['Accuracy', 'Precision', 'Recall', 'F1-Score', 'AUC-ROC']
x = np.arange(len(metrics))
width = 0.25

axes[0].bar(x - width, comparison_df.iloc[0, 1:], width, label='Regresión Logística', alpha=0.8)
axes[0].bar(x, comparison_df.iloc[1, 1:], width, label='Random Forest', alpha=0.8)
axes[0].bar(x + width, comparison_df.iloc[2, 1:], width, label='Deep Learning', alpha=0.8)

axes[0].set_xlabel('Métricas', fontsize=12)
axes[0].set_ylabel('Valor', fontsize=12)
axes[0].set_title('Comparación de Métricas entre Modelos', fontsize=14, fontweight='bold')
axes[0].set_xticks(x)
axes[0].set_xticklabels(metrics, rotation=45, ha='right')
axes[0].legend()
axes[0].grid(True, alpha=0.3, axis='y')
axes[0].set_ylim(0, 1)

# Gráfico de radar
angles = np.linspace(0, 2 * np.pi, len(metrics), endpoint=False).tolist()
angles += angles[:1]  # Cerrar el círculo

ax = plt.subplot(122, projection='polar')

for idx, row in comparison_df.iterrows():
    values = row[1:].tolist()
    values += values[:1]
    ax.plot(angles, values, 'o-', linewidth=2, label=row['Modelo'])
    ax.fill(angles, values, alpha=0.15)

ax.set_xticks(angles[:-1])
ax.set_xticklabels(metrics)
ax.set_ylim(0, 1)
ax.set_title('Comparación Radial de Métricas', fontsize=14, fontweight='bold', pad=20)
ax.legend(loc='upper right', bbox_to_anchor=(1.3, 1.1))
ax.grid(True)

plt.tight_layout()
plt.show()

## 9. Guardar el Modelo

In [ ]:
# Guardar el modelo entrenado
model.save('/content/drive/MyDrive/Credit Model/credit_scoring_deep_learning_model.h5')
print("Modelo guardado exitosamente en: credit_scoring_deep_learning_model.h5")

# Guardar el scaler para uso futuro
import pickle

with open('/content/drive/MyDrive/Credit Model/scaler.pkl', 'wb') as f:
    pickle.dump(scaler, f)
print("Scaler guardado exitosamente en: scaler.pkl")

# Guardar los label encoders
with open('/content/drive/MyDrive/Credit Model/label_encoders.pkl', 'wb') as f:
    pickle.dump(label_encoders, f)
print("Label encoders guardados exitosamente en: label_encoders.pkl")

## 10. Función de Predicción para Nuevos Datos

In [ ]:
def predict_credit_default(new_data, model, scaler, label_encoders):
    """
    Predice el riesgo de incumplimiento para nuevos datos.
    
    Args:
        new_data: DataFrame con las características del nuevo cliente
        model: Modelo de Deep Learning entrenado
        scaler: StandardScaler ajustado
        label_encoders: Diccionario con los LabelEncoders para variables categóricas
    
    Returns:
        Probabilidad de incumplimiento y clasificación binaria
    """
    # Crear una copia para no modificar los datos originales
    data = new_data.copy()
    
    # Eliminar Customer_ID si existe
    if 'Customer_ID' in data.columns:
        data = data.drop('Customer_ID', axis=1)
    
    # Codificar variables categóricas
    for col, le in label_encoders.items():
        if col in data.columns:
            data[col] = le.transform(data[col])
    
    # Escalar los datos
    data_scaled = scaler.transform(data)
    
    # Realizar la predicción
    probability = model.predict(data_scaled)[0][0]
    prediction = 1 if probability > 0.5 else 0
    
    return {
        'probabilidad_incumplimiento': probability,
        'prediccion': 'Incumplimiento' if prediction == 1 else 'No Incumplimiento',
        'score_crediticio': int((1 - probability) * 1000)  # Score de 0-1000
    }

print("Función de predicción definida.")
print("\nEjemplo de uso:")
print("resultado = predict_credit_default(nuevo_cliente_df, model, scaler, label_encoders)")

In [ ]:
# Ejemplo de predicción con un cliente del conjunto de prueba
sample_index = 0
sample_data = X_test.iloc[[sample_index]]

# Hacer la predicción
result = predict_credit_default(sample_data, model, scaler, label_encoders)

print("\n" + "="*60)
print("EJEMPLO DE PREDICCIÓN PARA UN NUEVO CLIENTE")
print("="*60)
print(f"Probabilidad de Incumplimiento: {result['probabilidad_incumplimiento']:.4f} ({result['probabilidad_incumplimiento']*100:.2f}%)")
print(f"Predicción: {result['prediccion']}")
print(f"Score Crediticio: {result['score_crediticio']}/1000")
print(f"\nValor Real: {'Incumplimiento' if y_test.iloc[sample_index] == 1 else 'No Incumplimiento'}")
print("="*60)

## 11. Conclusiones

### Resumen del Modelo

Este notebook implementa un modelo de Deep Learning para scoring crediticio utilizando una red neuronal profunda con las siguientes características:

**Arquitectura:**
- Red neuronal con 4 capas ocultas (128 → 64 → 32 → 16 neuronas)
- Batch Normalization para estabilizar el entrenamiento
- Dropout (30%-20%) para prevenir overfitting
- Activación ReLU en capas ocultas y Sigmoid en la salida

**Técnicas de Optimización:**
- Early Stopping para evitar sobreajuste
- Reducción adaptativa de learning rate
- Optimizador Adam con learning rate inicial de 0.001

**Ventajas del Modelo de Deep Learning:**
1. Capacidad para capturar relaciones no lineales complejas
2. Aprendizaje automático de features relevantes
3. Escalabilidad a grandes volúmenes de datos
4. Flexibilidad en la arquitectura

**Próximos Pasos Recomendados:**
1. Experimentar con diferentes arquitecturas (más/menos capas)
2. Probar diferentes técnicas de regularización
3. Implementar class weights para manejar el desbalance de clases
4. Realizar validación cruzada para mayor robustez
5. Análisis de features más importantes
6. Optimización de hiperparámetros con Grid Search o Bayesian Optimization

---
**Autor:** Claude  
**Fecha:** 2025  
**Versión:** 1.0
